# Adaptation Prompt Tuning

In this notebook, we will look into how to perform adaptation prompt tuning for generating short Text Ads.

Load the required libraries and the config parameters

In [ ]:
import os
os.environ["WANDB_PROJECT"]="prompt_learning_methods"
from enum import Enum
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed
from peft import get_peft_config, get_peft_model, AdaptionPromptConfig, TaskType, PeftType
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
seed = 42
set_seed(seed)
device = "cuda"
model_name_or_path = "meta-llama/Llama-2-7b-hf"
tokenizer_name_or_path = "meta-llama/Llama-2-7b-hf"

/raid/sourab/transformers/src/transformers/utils/hub.py:122: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-01-01 20:50:10.030221: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-01 20:50:10.030270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-01 20:50:10.031141: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-01 20:50:10.037212: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance

[2024-01-01 20:50:12,513] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


## Dataset Preparation

### Load the dataset

In [ ]:
from datasets import load_dataset
dataset_name = "jaykin01/advertisement-copy"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
template = """{% for message in messages %}\n{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% if loop.last and add_generation_prompt %}{{'<|im_start|>assistant\n' }}{% endif %}{% endfor %}"""
tokenizer.chat_template = template

system_prompt = """Create a text ad given the following product and description."""

def preprocess(samples):
    batch = []
    for product, desc, ad_copy in zip(samples["product"],samples["description"],samples["ad"]):
        conversation = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"""Product: {product}\nDescription: {desc}\n"""},
            {"role": "assistant", "content": f"""Ad: {ad_copy}\n"""},
        ]
        batch.append(tokenizer.apply_chat_template(conversation, tokenize=False))
    return {"content": batch}

dataset = load_dataset(dataset_name)
dataset = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset["train"].column_names
)

dataset["train"][0]

{'content': '<|im_start|>system\nCreate a text ad given the following product and description.<|im_end|>\n<|im_start|>user\nProduct:  Harem pants\nDescription:  A style of pants with a dropped crotch, loose-fitting legs, and a gathered waistband for a unique, bohemian look.\n<|im_end|>\n<|im_start|>assistant\nAd: Discover Harem Pants! Unique, stylish bohemian vibes with a dropped crotch & loose legs. Comfy meets chic - elevate your wardrobe. Limited stock - shop now!\n<|im_end|>\n'}

In [ ]:
dataset = dataset["train"].train_test_split(0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['content'],
        num_rows: 1026
    })
    test: Dataset({
        features: ['content'],
        num_rows: 115
    })
})

## Create the PEFT model

### Adaptation Prompt Tuning config

In [ ]:
peft_config = AdaptionPromptConfig(adapter_len=32,
                                   adapter_layers=30,
                                   task_type=TaskType.CAUSAL_LM)

In [ ]:
class ChatmlSpecialTokens(str, Enum):
    user = "<|im_start|>user"
    assistant = "<|im_start|>assistant"
    system = "<|im_start|>system"
    eos_token = "<|im_end|>"
    bos_token = "<s>"
    pad_token = "<pad>"

    @classmethod
    def list(cls):
        return [c.value for c in cls]
response_template = "<|im_start|>assistant\n"

tokenizer = AutoTokenizer.from_pretrained(
        model_name_or_path,
        pad_token=ChatmlSpecialTokens.pad_token.value,
        bos_token=ChatmlSpecialTokens.bos_token.value,
        eos_token=ChatmlSpecialTokens.eos_token.value,
        additional_special_tokens=ChatmlSpecialTokens.list(),
        trust_remote_code=True
    )
tokenizer.chat_template = template
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


In [ ]:
# creating model
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model.resize_token_embeddings(len(tokenizer))
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# cast non-trainable params in fp16
for p in model.parameters():
    if not p.requires_grad:
        p.data = p.to(torch.float16)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 3,932,190 || all params: 6,742,388,766 || trainable%: 0.05832042821127351


## Training

In [ ]:
output_dir = "llama_adcopy"
per_device_train_batch_size = 8
per_device_eval_batch_size = 8
gradient_accumulation_steps = 1
logging_steps = 5
learning_rate = 5e-4
max_grad_norm = 1.0
max_steps = 250
num_train_epochs=10
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
max_seq_length = 512

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_strategy="no",
    evaluation_strategy="epoch",
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    weight_decay=0.1,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    fp16=True,
    report_to=["tensorboard", "wandb"],
    hub_private_repo=True,
    push_to_hub=True,
    num_train_epochs=num_train_epochs,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False}
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    packing=False,
    dataset_text_field="content",
    max_seq_length=max_seq_length,
    data_collator=collator,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train()
trainer.save_model()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: smangrul. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss
1,1.459700,1.455904
2,0.895700,0.912726
3,0.692900,0.819112
4,0.665800,0.772601
5,0.644300,0.752461
6,0.685900,0.737771
7,0.616100,0.729294
8,0.625100,0.727424
9,0.645400,0.725144
10,0.578600,0.724965


/raid/sourab/peft/src/peft/utils/save_and_load.py:141: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

events.out.tfevents.1704138626.hf-dgx-01.2738331.0:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

In [ ]:
!nvidia-smi

## Loading the trained model and getting the predictions of the trained model

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch

peft_model_id = "Sanjaytfg/llama_adcopy"
device = "cuda"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
model.resize_token_embeddings(len(tokenizer))
model = PeftModel.from_pretrained(model, peft_model_id)


adapter_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [ ]:
model.to(torch.float16)
model.cuda()
model.eval()
messages = [
    {"role": "system", "content": "Create a text ad given the following product and description."},
    {"role": "user", "content": "Product: Sony PS5 PlayStation Console\nDescription: The PS5™ console unleashes new gaming possibilities that you never anticipated."},
]
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(text, return_tensors="pt")#, add_special_tokens=False)
inputs = {k: v.to("cuda") for k,v in inputs.items()}
outputs = model.generate(**inputs,
                         max_new_tokens=128,
                         do_sample=True,
                         top_p=0.95,
                         temperature=0.2,
                         repetition_penalty=1.1,
                         eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0]))

<s><|im_start|>system 
Create a text ad given the following product and description.<|im_end|> 
<|im_start|>user 
Product: Sony PS5 PlayStation Console
Description: The PS5™ console unleashes new gaming possibilities that you never anticipated.<|im_end|> 
<|im_start|>assistant 
Ad: Unlock your gaming potential with the PS5! 🎮🌟 Experience next-gen gaming and endless entertainment. Perfect for gamers and immersing yourself in epic worlds. Limited stock - game on! 🌟🕹️🏆
<|im_end|>


In [ ]:
!nvidia-smi